In [42]:
from pprint import pprint
import json

In [4]:
model_id, model_version, = (
    "huggingface-llm-falcon-7b-instruct-bf16",
    "*",
)

In [5]:
from ipywidgets import Dropdown

model_ids = [
    "huggingface-llm-falcon-40b-bf16",
    "huggingface-llm-falcon-40b-instruct-bf16",
    "huggingface-llm-falcon-7b-bf16",
    "huggingface-llm-falcon-7b-instruct-bf16",
]

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=model_ids,
    value=model_id,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(model_dropdown)

Dropdown(description='Select a model', index=3, layout=Layout(width='max-content'), options=('huggingface-llm-…

In [6]:
model_id = model_dropdown.value

In [7]:
model_id

'huggingface-llm-falcon-7b-instruct-bf16'

In [10]:
roleName = "SageMakerForSearch"

In [11]:
import sagemaker
import time 
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName=roleName)['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

Couldn't call 'get_role' to get Role ARN from role name Administrator to get Role path.


sagemaker role arn: arn:aws:iam::113170463366:role/SageMakerForSearch
sagemaker session region: us-east-1


In [14]:
%%time
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id,
                          role=role,
                          sagemaker_session=sess,)
predictor = my_model.deploy()

-----------------!Wall time: 9min 10s


In [16]:
%%time
prompt = "Tell me about Amazon SageMaker."
payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"],
    },
}

response = predictor.predict(payload)
print(response[0]["generated_text"])


Amazon SageMaker is a machine learning platform that allows users to train and deploy machine learning models quickly and easily. It offers a range of services, including automatic model tuning, feature selection, and model evaluation, which can help businesses and individuals make better decisions and create more accurate predictions.
Wall time: 3.71 s


In [17]:
prompt = """Answer based on context:\n\n{context}\n\n{question}"""

In [18]:
context = """
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics.
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992)."""

question = "Who won the gold in 2020 Summer Olympics men's high jump ?"

In [19]:
text_input = prompt.replace("{context}", context)
text_input = text_input.replace("{question}", question)

In [20]:
text_input

'Answer based on context:\n\n\nThe men\'s high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.\n33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places\nto enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).\nItalian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following\na tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance\nwhere the athletes of different nations had agreed to share the same medal in the history of Olympics.\nBarshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a\n\'jump off\'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men\'s high jump for Italy and

In [21]:
payload = {
    "inputs": text_input,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"],
    },
}

In [22]:
response = predictor.predict(payload)
print(response[0]["generated_text"])


The gold in 2020 Summer Olympics men's high jump was shared by Gianmarco Tamberi of Italy and Mutaz Essa Barshim of Qatar.


In [24]:
pprint(response[0]["generated_text"])

('\n'
 "The gold in 2020 Summer Olympics men's high jump was shared by Gianmarco "
 'Tamberi of Italy and Mutaz Essa Barshim of Qatar.')


In [49]:
context ="""
1 (a)  Radial symmetry\n'
 'Rationalised 2023-24\n'
 'ANIMAL KINGDOM 3939\n'
 '4.1.4 Coelom\n'
 'Presence or absence of a cavity between the body\n'
 'wall and the gut wall is very important in\n'
 'classification. The body cavity, which is lined\n'
 'by mesoderm is called coelom . Animals\n'
 'possessing coelom are called coelomates,  e.g.,\n'
 'annelids, molluscs, arthropods, echinoderms,\n'
 'hemichordates and chordates (Figure 4.3a)\n'
 '---\n'
 ' In\n'
 'some animals, the body cavity is not lined by\n'
 'mesoderm, instead, the mesoderm is present as\n'
 'scattered pouches in between the ectoderm and\n'
 'endoderm. Such a body cavity is called\n'
 'pseudocoelom and the animals possessing them\n'
 'are called pseudocoelomates,  e.g.,\n'
 'aschelminthes (Figure 4.3b). The animals in\n'
 'which the body cavity is absent are called\n'
 'acoelomates,  e.g., platyhelminthes (Figure 4.3c).Figure 4\n'
 '---\n'
 '6 Examples of  Coelenterata indicating outline of their body form :\n'
 '(a) Aurelia  (Medusa) (b) Adamsia (Polyp)\n'
 '(b) (a)Examples: Sycon  (Scypha), Spongilla (Fresh water sponge)  and '
 'Euspongia\n'
 '(Bath sponge).\n'
 '4.2.2 Phylum – Coelenterata (Cnidaria)\n'
 'They are aquatic, mostly marine, sessile or free-swimming, radially\n'
 'symmetrical animals (Figure 4.6). The name cnidaria is derived from the\n'
 'Rationalised 2023-24\n'
 '42 BIOLOGY\n'
 '4.2\n'
 '---\n'
 '3 Diagrammatic sectional view of :\n'
 '(a) Coelomate (b) Pseudocoelomate\n'
 '(c) AcoelomateThose animals in which the developing embryo has a thir d '
 'germinal layer ,\n'
 'mesoderm , in between the ectoderm and endoderm, are called\n'
 'triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).\n'
 '4.1.5 Segmentation\n'
 'In some animals, the body is externally and internally divided into\n'
 'segments with a serial repetition of at least some organs\n'
 '---\n
 """

In [50]:
question = "What is Coelom ?"

In [51]:
text_input = prompt.replace("{context}", context)
text_input = text_input.replace("{question}", question)

In [52]:
text_input

"Answer based on context:\n\n\n1 (a)  Radial symmetry\n'\n 'Rationalised 2023-24\n'\n 'ANIMAL KINGDOM 3939\n'\n '4.1.4 Coelom\n'\n 'Presence or absence of a cavity between the body\n'\n 'wall and the gut wall is very important in\n'\n 'classification. The body cavity, which is lined\n'\n 'by mesoderm is called coelom . Animals\n'\n 'possessing coelom are called coelomates,  e.g.,\n'\n 'annelids, molluscs, arthropods, echinoderms,\n'\n 'hemichordates and chordates (Figure 4.3a)\n'\n '---\n'\n ' In\n'\n 'some animals, the body cavity is not lined by\n'\n 'mesoderm, instead, the mesoderm is present as\n'\n 'scattered pouches in between the ectoderm and\n'\n 'endoderm. Such a body cavity is called\n'\n 'pseudocoelom and the animals possessing them\n'\n 'are called pseudocoelomates,  e.g.,\n'\n 'aschelminthes (Figure 4.3b). The animals in\n'\n 'which the body cavity is absent are called\n'\n 'acoelomates,  e.g., platyhelminthes (Figure 4.3c).Figure 4\n'\n '---\n'\n '6 Examples of  Coelenter

In [53]:
payload = {
    "inputs": text_input,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.1,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"],
    },
}

In [54]:
response = predictor.predict(payload)
print(response[0]["generated_text"])


Coelom is a body cavity present in animals that is lined by mesoderm. It is present between the body and the gut wall and is important in classifying animals.


In [55]:
pprint(response[0]["generated_text"])

('\n'
 'Coelom is a body cavity present in animals that is lined by mesoderm. It is '
 'present between the body and the gut wall and is important in classifying '
 'animals.')


In [56]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'hf-llm-falcon-7b-instruct-bf16-2023-08-12-14-37-19-114'

In [57]:
llm_endpoint = 'hf-llm-falcon-7b-instruct-bf16-2023-08-12-14-37-19-114'

In [58]:
def query_endpoint_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response

In [59]:
response2 = query_endpoint_with_json_payload(json.dumps(payload).encode("utf-8"), llm_endpoint)

In [60]:
model_predictions = json.loads(response2["Body"].read())

In [61]:
model_predictions[0]['generated_text']

'\nCoelom is a body cavity present in animals that is lined by mesoderm. It is a fluid-filled space located between the body and the gut wall. It is present in animals that are classified as coelomates.'